In [ ]:
import langchain
import langgraph
import langchain_core

Langraph, Builtin tools, custom tool

In [ ]:
from langchain_community.tools import WikipediaQueryRun

In [ ]:
from langchain_community.utilities import WikipediaAPIWrapper

In [ ]:
wiki_api = WikipediaAPIWrapper(top_k_results=5,doc_content_chars_max=100)

In [ ]:
wiki_tool = WikipediaQueryRun(api_wrapper=wiki_api)

In [ ]:
wiki_tool.run({"query":"Narendra Modi"})  # Example query to test the tool

In [ ]:
from langchain_community.tools import YouTubeSearchTool

Custom tool, cyclic graph

In [ ]:
def function1(input1):
    return input1+ " Prayag from first function"

In [ ]:
def function2(input2):
    return input2 + " Sonar from second function"

In [ ]:
def function3(input3):
    pass

In [ ]:
function1("prayag sonar")

In [ ]:
from langgraph.graph import StateGraph

In [ ]:
workflow1 = StateGraph(dict)

In [ ]:
workflow1.add_node("fun1", function1)

In [ ]:
workflow1.add_node("fun2",function2)

In [ ]:
workflow1.add_edge("fun1","fun2")

In [ ]:
workflow1.set_entry_point("fun1")

In [ ]:
workflow1.set_finish_point("fun2")

In [ ]:
app = workflow1.compile()

In [ ]:
from IPython.display import Image,display

In [ ]:
display(Image(app.get_graph().draw_mermaid_png()))

In [ ]:
app.invoke("This will display my graph")

In [ ]:
for output in app.stream("stream method invokation"):
    for key, value in output.items():
        print(f"here is the output from {key}")
        print("-----------------")
        print(value)
        print("\n")

Another custom too

In [ ]:
def llm(input):
    from langchain_google_genai import ChatGoogleGenerativeAI
    import os
    
    model = ChatGoogleGenerativeAI(model='gemini-1.5-flash')
    response = model.invoke(input)
    
    # Return the actual response content
    return response.content

In [ ]:
def token_counter(input):
    # Count tokens by splitting on whitespace
    tokens = input.split()
    token_count = len(tokens)
    
    # You can also count characters for a different metric
    char_count = len(input)
    
    return f"Response: '{input}'\nToken count (words): {token_count}\nCharacter count: {char_count}"

In [ ]:
workflow2 = StateGraph(dict)

In [ ]:
workflow2.add_node("My_LLM", llm)

In [ ]:
workflow2.add_node("LLM_output_token_counter",token_counter)

In [ ]:
workflow2.add_edge("My_LLM","LLM_output_token_counter")

In [ ]:
workflow2.set_entry_point("My_LLM")

In [ ]:
workflow2.set_finish_point("LLM_output_token_counter")

In [ ]:
app = workflow2.compile()

In [ ]:
display(Image(app.get_graph().draw_mermaid_png()))

In [ ]:
app.invoke("can you tell me about the india capital")

In [ ]:
def llm(input):
    from langchain_google_genai import ChatGoogleGenerativeAI
    import os
    
    # Make sure to set your Google API key
    # os.environ["GOOGLE_API_KEY"] = "your_api_key_here"
    
    model = ChatGoogleGenerativeAI(model='gemini-1.5-flash')
    response = model.invoke(input)
    
    # Return the actual response content
    return response.content

In [ ]:
# Set up your Google API key
import os
from getpass import getpass

# Uncomment and use one of these methods:
# Method 1: Set directly (not recommended for production)
# os.environ["GOOGLE_API_KEY"] = "your_api_key_here"

# Method 2: Load from environment file
# from dotenv import load_dotenv
# load_dotenv()

# Method 3: Enter securely (recommended for testing)
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass("Enter your Google API key: ")

In [ ]:
def token_counter(input):
    # Count tokens by splitting on whitespace
    tokens = input.split()
    token_count = len(tokens)
    
    # You can also count characters for a different metric
    char_count = len(input)
    
    return f"Response: '{input}'\nToken count (words): {token_count}\nCharacter count: {char_count}"

In [ ]:
# Advanced token counter using Google's tokenization
def advanced_token_counter(input_text):
    try:
        import google.generativeai as genai
        
        # Configure with your API key
        genai.configure(api_key=os.environ.get("GOOGLE_API_KEY"))
        
        # Create a model instance for token counting
        model = genai.GenerativeModel('gemini-1.5-flash')
        
        # Count tokens
        token_count = model.count_tokens(input_text)
        
        return f"Text: '{input_text}'\nActual tokens (Gemini): {token_count.total_tokens}\nWord count: {len(input_text.split())}"
        
    except Exception as e:
        # Fallback to simple word counting
        word_count = len(input_text.split())
        return f"Text: '{input_text}'\nWord count (fallback): {word_count}\nError: {str(e)}"

In [ ]:
# Recompile the workflow with updated functions
workflow2 = StateGraph(dict)
workflow2.add_node("My_LLM", llm)
workflow2.add_node("LLM_output_token_counter", token_counter)
workflow2.add_edge("My_LLM", "LLM_output_token_counter")
workflow2.set_entry_point("My_LLM")
workflow2.set_finish_point("LLM_output_token_counter")

# Compile the updated workflow
app2 = workflow2.compile()

print("Workflow recompiled with updated functions!")

In [ ]:
# Test the updated workflow
print("Testing the updated workflow:")
print("=" * 50)

result = app2.invoke("Can you tell me about India's capital?")
print("Final result:", result)